In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data = pd.read_csv("data.tsv", skipinitialspace = True, sep = "\t")

data.head()

,Kooli ID,Kooli nimi,Maakond,Asutuse tüüp,Seadmete nimekiri,Toetus 2014,Toetus 2015,Toetus 2016,Toetus 2017,Toetus 2018,Toetus 2019,Toetus 2020,Toetus 2021
0,437,Abja Gümnaasium,Viljandi maakond,gümnaasium,"LEGO Mindstorm EV3 baaskomplekt (9 tk), lisako...","3,000.00 €",NaN,NaN,NaN,NaN,"2,154.24 €",NaN,NaN
1,335,Antsla Gümnaasium,Võru maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (4 tk), EV3 l...",NaN,"1,279.20 €",NaN,NaN,"1,589.00 €",NaN,NaN,NaN
2,199,Aravete Keskkool,Järva maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (8 tk), EV3 l...",NaN,"3,535.26 €",NaN,NaN,"1,880.35 €",NaN,"2,857.68 €","4,410 €"
3,256,August Kitzbergi nimeline Gümnaasium,Viljandi maakond,gümnaasium,Kodulabor III (1 tk) ja LEGO Mindstorms EV3 b...,"1,151.40 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,597,Avinurme Gümnaasium,Jõgeva maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (3 tk), EV3 l...",NaN,"1,146.53 €",NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#Translations of column names:

#Kooli ID - school's ID
#Maakond - county
#Asutuse tüüp - Type of school (high school, middle school, vocational school etc)
#Seadmete nimekiri - list of devices
#Toetus - funding

In [3]:
#basic check for possible typos in some columns
print("Maakonnad:")
data['Maakond'].value_counts()
print("Asutused:")
data['Asutuse tüüp'].value_counts()

Maakonnad:


Harju maakond         201
Tartu maakond          96
Ida-Viru maakond       64
Pärnu maakond          60
Lääne-Viru maakond     41
Viljandi maakond       33
Rapla maakond          32
Võru maakond           28
Jõgeva maakond         25
Järva maakond          24
Põlva maakond          23
Lääne maakond          22
Saare maakond          18
Valga maakond          15
Hiiu maakond            9
Name: Maakond, dtype: int64

Asutused:


lasteaed                    301
põhikool                    196
gümnaasium                  124
lasteaed-põhikool            59
kutseõppeasutus              10
huvikool                      1
täiskasvanute gümnaasium      1
Name: Asutuse tüüp, dtype: int64

In [24]:
import re
    
alldevices = []
for i in range(len(data)):
    devicelist = str(data.iloc[i,4])
    devicesfromrow = re.split(";", devicelist)
    for device in devicesfromrow:
        if not device == "": 
            alldevices.append(device.strip())

alldevicesdf = pd.DataFrame(alldevices)
#alldevicesdf.head()
#alldevicesdf.value_counts()
pd.set_option('display.max_rows', alldevicesdf.shape[0]+1)
alldevicesdf.sort_values(by=0)


,0
2726,
2723,
1998,
690,
2878,
2380,"""Lepatriinu"" ""robot kit elektroonika-mehaanika..."
3137,+ lisatarvikud
285,10 freesitera komplekt (1tk)
101,10 freesitera komplekt (1tk)
354,10 freesitera komplekt (2 tk)
